In [ ]:
%matplotlib inline
import sys
import os
import numpy
import struct
sys.path.append(os.path.expanduser("~/jng/sunshine/noddy"))

In [ ]:
from karmapi import weather
from datetime import date, timedelta

In [ ]:
ls -al ../*.gz

In [ ]:
ls -al

In [ ]:
data = weather.get_data(date(1979,1,1), open('tmax.csv'))

In [ ]:
def tonumpy(data):
    
    ndata = numpy.array([float(x) for x in data.split()])
    ndata = ndata.reshape(weather.longitudes(), weather.latitudes()).T
    return ndata

In [ ]:
from matplotlib import pyplot

In [ ]:
data = tonumpy(data)
pyplot.imshow(data)

In [ ]:
!mkdir euro

In [ ]:
# mmake some directories
delta = weather.DELTA
lon = weather.LONGITUDE_START
for ix in range(weather.longitudes()):
    folder = 'euro/{}'.format(lon)
    
    try:
        os.makedirs(folder) 
    except:
        pass
    
    lat = weather.LATITUDE_START
    for ix in range(weather.latitudes()):
    
        folder = 'euro/{}/{}'.format(lon, lat)
    
        try:
            os.makedirs(folder)
        except:
            pass
        
        lat -= delta
    lon += delta

In [ ]:
pyplot.plot(data[:,30])

In [ ]:
# open a file for each lon
lons = numpy.linspace(weather.LONGITUDE_START, 360.0, weather.longitudes())

lons = !ls euro

outfiles = [open('euro/{}/tmax'.format(x), 'ab') for x in lons]

In [ ]:
def write_day(data, date, outfiles):

    packer = struct.Struct('{}f'.format(weather.latitudes()))

    for ix in range(weather.longitudes()):
    
        col = data[:, ix]
        pdata = packer.pack(*col)
    
        outfiles[ix].write(pdata)

    

In [ ]:
def process_days(start, end, infile, outfiles):
    
    day = start
    while day < end:
        if day.day == 1: print(day)
        data = tonumpy(weather.get_data(day, infile))
        
        write_day(data, date, outfiles)
        
        day += timedelta(days=1)


In [ ]:
infile = open('tmax.csv')
process_days(date(1989,1,1), date(2016,1,1), infile, outfiles)

In [ ]:
for out in outfiles: out.close()

In [ ]:
# get data for a lat lon
def lat_lon(lat, lon, value='tmax'):
    
    # read all data for lon
    infile = "euro/{lat}/{value}".format(**locals())
    
    data = open(infile, 'rb').read()
    
    print(len(data)/4)
    
    unpack = struct.Struct('{}f'.format(int(len(data)/4)))
    
    return unpack.unpack(data)


In [ ]:
data = lat_lon(0.0, 0.0)

In [ ]:
d = data[170::241]

In [ ]:
pyplot.plot(d)